In [2]:
import json
from pathlib import Path
from openslide import OpenSlide
from PIL import Image


def split_tif_image_with_position(tif_path, output_dir, metadata_path, tile_size=1024, overlap=0.2, level=0):
    """
    Split a TIF image into smaller tiles and record their positions using OpenSlide.
    
    Parameters:
    - tif_path: str, path to the TIF file.
    - output_dir: str, directory to save the tiles.
    - metadata_path: str, path to save the metadata JSON file.
    - tile_size: int, size of each tile (default 1024x1024).
    - overlap: float, overlap ratio between tiles (default 20%).
    - level: int, the resolution level to read from the TIF file (default 0 for full resolution).
    """
    # Open the TIF file using OpenSlide
    slide = OpenSlide(tif_path)
    
    # Get the dimensions of the image at the specified level
    width, height = slide.level_dimensions[level]
    
    # Create output directory
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Step size with overlap
    step = int(tile_size * (1 - overlap))

    # Metadata to store tile positions
    metadata = {}

    # Iterate through the image and create tiles
    for x in range(0, width, step):
        for y in range(0, height, step):
            # Define the cropping box
            box = (x, y, min(x + tile_size, width), min(y + tile_size, height))
            
            # Read the region from the slide
            region = slide.read_region(
                location=(box[0], box[1]), 
                level=level, 
                size=(box[2] - box[0], box[3] - box[1])
            )
            
            # Convert region to RGB mode if necessary
            tile = region.convert("RGB")
            
            # Generate tile filename
            tile_filename = f"tile_{x}_{y}.png"
            tile_filepath = output_dir / tile_filename
            
            # Save tile image
            tile.save(tile_filepath)
            
            # Record metadata for this tile
            metadata[tile_filename] = {
                "x_start": box[0],
                "y_start": box[1],
                "x_end": box[2],
                "y_end": box[3]
            }

    # Save metadata to JSON file
    with open(metadata_path, "w") as f:
        metadata_path = output_dir / metadata_path
        json.dump(metadata, f, indent=4)

    print(f"Splitting completed! Tiles saved to {output_dir}, metadata saved to {metadata_path}")


# Example usage
tif_path = "G:/241001_0012024-10-14_11_57_29.tif"  # Path to the TIF file
output_dir = "G:/output_tiles"    # Directory to save tiles
metadata_path = "tile_metadata.json"  # Path to save metadata

split_tif_image_with_position(
    tif_path=tif_path,
    output_dir=output_dir,
    metadata_path=metadata_path,
    tile_size=1024,
    overlap=0.2,
    level=0
)


Splitting completed! Tiles saved to G:\output_tiles, metadata saved to tile_metadata.json
